In [7]:
# This function is to read the cvs embeddings create from colab
import random

import torch
import numpy as np 
import pandas as pd


# Import texts and embedding df
file_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embedding_df = pd.read_csv(file_path)



In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [5]:
text_chunks_and_embedding_df.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00,[ 6.74242675e-02 9.02281404e-02 -5.09548886e-...
1,-38,Human Nutrition: 2020 Edition by University of...,210,30,52.50,[ 5.52156419e-02 5.92139773e-02 -1.66167244e-...
2,-37,Contents Preface University of Hawai‘i at Māno...,766,114,191.50,[ 2.79801842e-02 3.39813754e-02 -2.06426680e-...
3,-36,Lifestyles and Nutrition University of Hawai‘i...,941,142,235.25,[ 6.82566911e-02 3.81275006e-02 -8.46854132e-...
4,-35,The Cardiovascular System University of Hawai‘...,998,152,249.50,[ 3.30264494e-02 -8.49763490e-03 9.57159605e-...


In [9]:
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

# Convert texts and embedding df to list of dicts
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

# Convert embeddings to torch tensor and send to device (note: NumPy arrays are float64, torch tensors are float32 by default)
embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings.shape

torch.Size([1680, 768])

In [11]:
text_chunks_and_embedding_df["embedding"][0]

array([ 6.74242675e-02,  9.02281404e-02, -5.09548886e-03, -3.17545570e-02,
        7.39082173e-02,  3.51976119e-02, -1.97986923e-02,  4.67692539e-02,
        5.35727106e-02,  5.01231011e-03,  3.33929174e-02, -1.62219570e-03,
        1.76080856e-02,  3.62653658e-02, -3.16683349e-04, -1.07118404e-02,
        1.54258069e-02,  2.62176115e-02,  2.77654896e-03,  3.64942513e-02,
       -4.44109552e-02,  1.89361926e-02,  4.90117930e-02,  1.64020136e-02,
       -4.85782810e-02,  3.18290642e-03,  2.72992700e-02, -2.04758439e-03,
       -1.22828931e-02, -7.28049129e-02,  1.20446077e-02,  1.07300160e-02,
        2.10001227e-03, -8.17772895e-02,  2.67830114e-06, -1.81428716e-02,
       -1.20803080e-02,  2.47174613e-02, -6.27467185e-02,  7.35438913e-02,
        2.21624430e-02, -3.28768007e-02, -1.80095788e-02,  2.22952422e-02,
        5.61365187e-02,  1.79511297e-03,  5.25931939e-02, -3.31743481e-03,
       -8.33873916e-03, -1.06284320e-02,  2.31919112e-03, -2.23934129e-02,
       -1.53011940e-02, -

In [14]:
from sentence_transformers import util, SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", 
                                      device="cpu") # choose the device to load the model to

ImportError: cannot import name 'fastCopyAndTranspose' from 'numpy.core._multiarray_umath' (c:\Users\leeju\OneDrive - Pesonal\OneDrive - Nanyang Technological University\Desktop\RAGNutri\.venv\lib\site-packages\numpy\core\_multiarray_umath.py)

In [ ]:
# 1. Define the query
# Note: This could be anything. But since we're working with a nutrition textbook, we'll stick with nutrition-based queries.
query = "macronutrients functions"
print(f"Query: {query}")

# 2. Embed the query to the same numerical space as the text examples 
# Note: It's important to embed your query with the same model you embedded your examples with.
query_embedding = embedding_model.encode(query, convert_to_tensor=True)

# 3. Get similarity scores with the dot product (we'll time this for fun)
from time import perf_counter as timer

start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
end_time = timer()

print(f"Time take to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

# 4. Get the top-k results (we'll keep this to 5)
top_results_dot_product = torch.topk(dot_scores, k=5)
top_results_dot_product